In [0]:
CREATE TABLE IF NOT EXISTS gold.dim_customer(
  customer_sk BIGINT GENERATED ALWAYS AS IDENTITY,
  customer_id STRING,
  customer_unique_id STRING,
  customer_zip_code_prefix STRING,
  customer_city STRING,
  customer_state STRING,
  geolocation_sk BIGINT,
  start_date DATE,
  end_date DATE,
  is_current BOOLEAN
) USING DELTA;


In [0]:
%python
from pyspark.sql.functions import current_date, lit

silver_df = spark.read.table("silver.customers_cleaned") 

updates_df = (
    silver_df
    .withColumn("start_date", current_date())
    .withColumn("end_date", lit(None).cast("date"))
    .withColumn("is_current", lit(True))
)
updates_df.display()

In [0]:
%python
updates_df.createOrReplaceTempView("updates_view")

In [0]:
MERGE INTO dim_customer AS target
USING updates_view AS source
ON target.customer_id = source.customer_id AND target.is_current = true

WHEN MATCHED AND (
    target.customer_city != source.customer_city OR
    target.customer_state != source.customer_state
) THEN
  UPDATE SET
    end_date = current_date(),
    is_current = false;

MERGE INTO dim_customer AS target
USING updates_view AS source
ON target.customer_sk = source.customer_sk AND target.is_current = true
WHEN NOT MATCHED THEN
  INSERT (
    customer_id,
    customer_unique_id,
    customer_zip_code_prefix,
    customer_city,
    customer_state,
    geolocation_sk,
    start_date,
    end_date,
    is_current
  )
  VALUES (
    source.customer_id,
    source.customer_unique_id,
    source.customer_zip_code_prefix,
    source.customer_city,
    source.customer_state,
    source.geolocation_sk,
    current_date(),
    NULL,
    true
  );


In [0]:
SELECT * FROM gold.dim_customer LIMIT 10;